In [21]:
import pandas as pd

train_data = pd.read_csv('../data/train_processed.csv', index_col=0)
test_data = pd.read_csv('../data/test_processed.csv', index_col=0)

In [25]:
train_data.head(50)

SiteId                   0
ForecastId               0
Value                86601
wday                     0
mday                     0
yday                     0
month                    0
year                     0
time                     0
wday_sin                 0
wday_cos                 0
yday_sin                 0
yday_cos                 0
month_sin                0
month_cos                0
time_sin                 0
time_cos                 0
off                      0
BaseTemperature          0
Surface                  0
Temperature        4318750
Distance           4318750
dtype: int64

In [10]:
def process_data(data: pd.DataFrame):
    labels = data['Value']
    df = data.drop(columns=['ForecastId', 'Distance'])

    return df, labels

train_x, train_y = process_data(train_data)
test_x, test_y = process_data(test_data)

In [18]:
train_x.isna().sum()

SiteId                   0
Value                86601
wday                     0
mday                     0
yday                     0
month                    0
year                     0
time                     0
wday_sin                 0
wday_cos                 0
yday_sin                 0
yday_cos                 0
month_sin                0
month_cos                0
time_sin                 0
time_cos                 0
off                      0
BaseTemperature          0
Surface                  0
Temperature        4318750
dtype: int64

## Linear Regression

In [16]:
from sklearn.linear_model import LinearRegression

In [17]:
linear_model = LinearRegression()
linear_model.fit(train_x, train_y)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor